In [93]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pymongo
from pprint import pprint

In [94]:
#urls to scrape
nasamarsurl = "https://mars.nasa.gov/news/"
nasajplurl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
sfurl = "https://space-facts.com/mars/"
usgsurgl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# connect to the web with splinter
driverpath = {'executable_path':'Resources/chromedriver.exe'}

#connect to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#database paths
nasamars_collection = client.mission_to_mars.nasamars
jplimages_collection = client.mission_to_mars.jplimages


In [115]:
def nasamars_scraper(scrapeurl,collection):
    ''''''
    #scrape NASA Mars News articles
    def pull_data(scrapeurl):
       
        browser = Browser('chrome',**driverpath, headless=False)
        browser.visit(scrapeurl)
        rawdata = browser.html
        browser.quit()
        
        return rawdata
    
    def clean_data(rawdata,collection):
        # clear database to prevent duplicate entries
        collection.drop()

        #parse the sites html code into a searchable list
        soup = bs(rawdata, 'html.parser')

        # put the html structure for each article into a list
        content = (soup
                    .find('section', class_='grid_gallery module list_view')
                    .find_all('div', class_= 'list_text'))

        for entry in content:
        # iterate through the list and return desired attributes for each article
            try:
                #extract desired attributes
                title = (entry.find('a').text)
                summary = (entry.find('div', class_= 'article_teaser_body').text)
                date = (entry.find('div', class_ = 'list_date').text)
                url = entry.find('a','href')

                #save to mongoDB
                document = {
                    'title': title,
                    'summary': summary,
                    'date': date,
                    'url': url
                }
                #print(f"Saved Article: {document['title']}")
                collection.insert_one(document)

            except Exception as error:
                print(error)
                
    rawdata = pull_data(scrapeurl)
    return clean_data(rawdata,collection)

In [116]:
nasamars_scraper(nasamarsurl,nasamars_collection)

Saved Article: NASA's Perseverance Rover Begins Its First Science Campaign on Mars
Saved Article: NASA's InSight Mars Lander Gets a Power Boost
Saved Article: NASA's Curiosity Rover Captures Shining Clouds on Mars
Saved Article: Life Goals: NASA Software Unlocks Martian Rover Productivity
Saved Article: Salts Could Be Important Piece of Martian Organic Puzzle
Saved Article: Seeing NASA's Ingenuity Mars Helicopter Fly in 3D
Saved Article: Perseverance's Robotic Arm Starts Conducting Science
Saved Article: NASA's Ingenuity Mars Helicopter Completes First One-Way Trip  
Saved Article: NASA's Perseverance Captures Video, Audio of Fourth Ingenuity Flight
Saved Article: NASA's Ingenuity Helicopter to Begin New Demonstration Phase
Saved Article: NASA Sets Briefing to Discuss Ingenuity Mars Helicopter Next Steps
Saved Article: With Goals Met, NASA to Push Envelope With Ingenuity Mars Helicopter
Saved Article: NASA's Ingenuity Mars Helicopter Flies Faster, Farther on Third Flight
Saved Article:

In [56]:
test = {'seven':1+22,'nine':93+1}

test
document = {f'{test.keys()}':test}
document

#for blah, value in test.items():
  #  print(f'Key:{blah} Value:{value}')

{"dict_keys(['seven', 'nine'])": {'seven': 23, 'nine': 94}}

In [ ]:
test = {'k1':"v1","k2":"v2"}

attributes
document = {f'{attributes.key()}':attributes.value()}
document